# Tendon-driven simulation
This notebook provides the necessary function to run the mujoco tendon-driven simulation of the tail.

In [6]:
import mujoco
import mediapy as media
from mujoco import viewer
import matplotlib.pyplot as plt
import numpy as np

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

model = mujoco.MjModel.from_xml_path(r"xml\fullSizeAndBoxes.xml")
renderer = mujoco.Renderer(model, 480, 640)

def tendon_actuation(data, tendon_command, ctrl, step, factor):
  """Function for actuating the tendons

  Args:
      data (struct): Data of the model
      tendon_command (float): Desired command
      ctrl (float): Current command
      step (float): Step of control change
      factor (float): Factor of power

  Returns:
      float: New command
  """
  if tendon_command == "left" and ctrl> -1:
    ctrl = ctrl - step*factor

  elif tendon_command == "right" and ctrl<1:
    ctrl = ctrl+step
    
  elif tendon_command == "stop":
    if ctrl >= step:
      ctrl -=step
    elif ctrl <= -step:
      ctrl +=step

  data.actuator('muscle1').ctrl = ctrl
  data.actuator('muscle2').ctrl = -ctrl
  return ctrl

def body_rotation(data, pos, vel,des_pos = 0.52, max_vel = 5):
  """Function to actuate the body rotation with a controller.

  Args:
      data (struct): Data of the model
      pos (float): Current position
      vel (float): Current velocity
      des_pos (float, optional): Desired position. Defaults to 0.52.
      max_vel (int, optional): Maximum velocity. Defaults to 5.
  """
  kp = 1.5
  kb = 0.1
  kv = 1
  if max_vel == 0:
    data.actuator('J0').ctrl = 0

  elif abs(vel) > 0.9*abs(max_vel):
    data.actuator('J0').ctrl = -kb*(0.9 + vel/max_vel)
  else:
    data.actuator('J0').ctrl = -kp*(-des_pos + pos)- kv*(vel/(0.9*max_vel))

def plot_graph(times, veltip, velcm, velang, gyro, body_angle, ctrl):
  """Function to plot the graphs of the experiment's data

  Args:
      times (list): Time values
      veltip (list): Velocity of the tip of the club in m/s
      velcm (list): Velocity of the club's CM in m/s
      velang (list): Angular velocity of the body in rad/s
      gyro (list): Angle of the body in rad
      body_angle (list): Angle of the body
      ctrl (list): Tendon's command
  """
  dpi = 120
  width = 640
  height = 1000
  figsize = (width / dpi, height / dpi)
  fig, ax = plt.subplots(6, figsize=figsize)
  ax[0].plot(times, veltip)
  ax[1].plot(times, velcm)
  ax[2].plot(times, velang)
  ax[3].plot(times, gyro)
  ax[4].plot(times, body_angle)
  ax[5].plot(times, ctrl)

def simulation(blow_timer = 0.4, body_rot_offset = 0.0, body_rot = 0.52, rot_speed = 5, tendon_step = 1, factor = 1, fps = 60, duration = 3, model = model, video = False, plot= False, camera = "closeup"):
  """Run a simulation instance with the chosen parameters. Details about each parameters can be found in the report.

  Args:
      blow_timer (float, optional): Swing offset in seconds. Defaults to 0.4.
      body_rot_offset (float, optional): Body rotation offset in seconds. Defaults to 0.0.
      body_rot (float, optional): Body rotation in rad. Defaults to 0.52.
      rot_speed (int, optional): Body rotation speed in rad/s. Defaults to 5.
      tendon_step (int, optional): Tendon-step. Defaults to 1.
      factor (int, optional): _description_. Defaults to 1.
      fps (int, optional): _description_. Defaults to 60.
      duration (int, optional): Duration of the instance in seconds. Defaults to 3.
      model (_type_, optional): Model used for the simulation. Defaults to model.
      video (bool, optional): If True, render a video of the simulation. Defaults to False.
      plot (bool, optional): If true, plot some useful values. Defaults to False.
      camera (str, optional): Chose which camera to use for the rendering, there is two camera: "closeup" and "topview". Defaults to "closeup".

  Returns:
      _type_: _description_
  """
  data = mujoco.MjData(model)
  ctrl_tendon = 0; phase = 0; delay = 0; release_delay = 50; F = []
  tendon_command = "stop"
  times =[]; veltip =[]; velcm =[]; gyro =[]; frames = []; body_angle =[]; ctrl = []; velang = []
  E = 0
  pE = 0
  E_body = 0
  n_frames = int(duration*fps)

  for i in range(n_frames):
    while data.time < i/fps:
      mujoco.mj_step(model, data)
      if phase ==0: #preblow
        tendon_command = "left"
        des_body_rot = -body_rot/2
        phase +=1
        
      elif data.time > blow_timer and phase ==1:
        if body_rot_offset > 0:
          tendon_command = "right"
          phase+=1
        elif body_rot_offset < 0:
          des_body_rot = body_rot
          phase+=1
        else:
          des_body_rot = body_rot
          tendon_command = "right"
          phase+=2

      elif data.time > blow_timer+abs(body_rot_offset) and phase ==2:
        if body_rot_offset > 0:
          des_body_rot = body_rot
          phase+=1
        elif body_rot_offset < 0:
          tendon_command = "right"
          phase+=1
        else:
          print("Error")

      if phase ==3:
        if delay < release_delay:
          delay += 1
        elif data.qvel[1]+data.qvel[2]+data.qvel[3]+data.qvel[4]+data.qvel[5]+data.qvel[6]+data.qvel[7] >= -1:
          tendon_command ="stop"
          
          
      ctrl_tendon = tendon_actuation(data, tendon_command, ctrl_tendon, tendon_step, factor)
      body_rotation(data, data.qpos[0], data.qvel[0], des_body_rot, rot_speed)
      times.append(data.time)
      veltip.append(np.sqrt(data.sensor('gpstip').data[0].copy()**2+data.sensor('gpstip').data[1].copy()**2+data.sensor('gpstip').data[2].copy()**2)*np.sign(+data.sensor('gpstip').data[1].copy()))
      velcm.append(np.sqrt(data.sensor('gpscm').data[0].copy()**2+data.sensor('gpscm').data[1].copy()**2+data.sensor('gpscm').data[2].copy()**2))
      velang.append(-data.qpos[7])
      gyro.append(-data.qvel[0])
      body_angle.append(-(data.qpos[0]))
      ctrl.append(ctrl_tendon)
      F.append(np.sqrt(data.sensor('tendonForce').data[0].copy()**2+data.sensor('tendonForce').data[1].copy()**2+data.sensor('tendonForce').data[2].copy()**2))
      #compute the energy of the body
      pE = E
      E = 1/2*320*data.qvel[0]**2
      if E-pE > 0:
          E_body += E-pE
    renderer.update_scene(data, camera = camera)
    frame = renderer.render()
    frames.append(frame)
  if(video):# Simulate and display video.
    media.show_video(frames, fps=fps)
  #calculate energy of the tail
  E_tail = 0.05*1.13*(max(ctrl)-min(ctrl))*np.mean(F)
  if(plot):
    plot_graph(times, veltip, velcm, velang, gyro, body_angle, ctrl)
  return veltip, gyro, E_tail, E_body

ValueError: Error: could not parse OBJ file 'xml\..\..\Obj\Club doedicurus_final_version_full_size.obj': 
Cannot open file [xml\..\..\Obj\Club doedicurus_final_version_full_size.obj]

Object name = clubMesh, id = 0, line = 23, column = -1

In [3]:
experiment = []

xml5deg = "C:\\Users\\olonn\\Desktop\\PDM\Simulations\\mujoco full size\\5deg.xml"
xml7deg = "C:\\Users\\olonn\\Desktop\\PDM\\Simulations\\mujoco full size\\7deg.xml"
xml12deg ="C:\\Users\\olonn\\Desktop\\PDM\Simulations\\mujoco full size\\12deg.xml"
xml15deg = "C:\\Users\\olonn\\Desktop\\PDM\\Simulations\\mujoco full size\\15deg.xml"

model = mujoco.MjModel.from_xml_path(xml5deg)
experiment.append(simulation(model = model))
model = mujoco.MjModel.from_xml_path(xml7deg)
experiment.append(simulation(model = model))
model = mujoco.MjModel.from_xml_path(xml12deg)
experiment.append(simulation(model = model))
model = mujoco.MjModel.from_xml_path(xml15deg)
experiment.append(simulation(model = model))

for i in range(0, 9):
    blow_timer = 0.1*i
    experiment.append(simulation(blow_timer = blow_timer))

for i in range(0, 7):
    body_rot = 0.1745*i
    experiment.append(simulation(body_rot = body_rot))

for i in range(0,9):
    body_rot_offset = -0.4 +0.1*i
    experiment.append(simulation(body_rot_offset= body_rot_offset))

for i in range(0,10):
    rot_speed = 1 +i
    experiment.append(simulation(rot_speed = rot_speed))

for i in range(0,10):
    factor = 0.1*i +0.1
    experiment.append(simulation(factor = factor))
